In [1]:
import logging
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s - %(levelname)s - %(filename)s:%(lineno)d - %(message)s",
)

from app.instructions import global_tools_hub
global_tools_hub.load_tools("tools")

2025-02-18 20:11:55,886 - INFO - tools.py:82 - Loading module report_utils from report_utils.py
2025-02-18 20:11:55,936 - INFO - tools.py:82 - Loading module smart_retrieve from smart_retrieve.py
2025-02-18 20:11:56,444 - INFO - smart_retrieve.py:214 - Using deepseek-r1-distill-qwen-32b Reasoning LLM
2025-02-18 20:11:56,456 - INFO - smart_retrieve.py:216 - Using gpt-4o Evaluation LLM
2025-02-18 20:11:56,456 - INFO - tools.py:104 - Registered tool 'retrieve_knowledge_graph' from smart_retrieve.py
2025-02-18 20:11:56,457 - INFO - tools.py:82 - Loading module llm from llm.py
2025-02-18 20:11:56,471 - INFO - tools.py:104 - Registered tool 'llm_generate' from llm.py
2025-02-18 20:11:56,471 - INFO - tools.py:82 - Loading module retrieve from retrieve.py
2025-02-18 20:11:56,480 - INFO - tools.py:104 - Registered tool 'vector_search' from retrieve.py


In [ ]:
from notebooks.tasks import get_evaluation_pending_tasks
from datetime import datetime, timedelta

end_time = datetime.utcnow()
start_time = end_time - timedelta(hours=4)

pending_tasks = get_evaluation_pending_tasks(
    start_time=start_time,
    statuses=["REJECTED"]
)

pending_tasks = [t for t in pending_tasks if t['status'] == "completed" and t['human_evaluation_status'] == "WAITING_FOR_EVALUATION"]
pending_tasks

In [2]:
from notebooks.plan_mcts_optimizer import MCTSPlanOptimizer

# task_id = pending_tasks[0]['id']
task_id = "e7a7c4d4-bbfd-47d1-9375-0e821032e122"

optimizer = MCTSPlanOptimizer(task_id=task_id, max_iterations=3, time_limit_seconds=900)

2025-02-18 20:12:17,590 - INFO - base.py:507 - Adding job tentatively -- it will be properly scheduled when the scheduler starts
2025-02-18 20:12:17,590 - INFO - base.py:1090 - Added job "SimpleCache.refresh_cache" to job store "default"
2025-02-18 20:12:17,591 - INFO - base.py:214 - Scheduler started
2025-02-18 20:12:17,591 - INFO - simple_cache.py:40 - Started cache refresh scheduler to run every 24 hours.
2025-02-18 20:12:17,592 - INFO - simple_cache.py:131 - Starting cache refresh...
2025-02-18 20:12:18,010 - INFO - simple_cache.py:165 - Cache refresh completed successfully.
2025-02-18 20:12:18,824 - INFO - plan_mcts_optimizer.py:336 - add child node: 58082dd4aeee4a89870817fb11247611(-1) ---> 762408de52714372bb777449b7f625db(0)
2025-02-18 20:12:18,825 - INFO - plan_mcts_optimizer.py:336 - add child node: 762408de52714372bb777449b7f625db(0) ---> 282930766af54277b8a0ed271db6543c(1)
2025-02-18 20:12:18,826 - INFO - plan_mcts_optimizer.py:336 - add child node: 282930766af54277b8a0ed271

In [3]:
print("optimizing task", task_id)

def print_node(node):
    print("*" * 100)
    print(node.state.seq_no, node.state.commit_hash, node.visits, node.value)
    for suggestion in node.optimization_suggestions:
        print(suggestion["branch_name"])
        print(suggestion["suggestion"])
    for child in node.children:
        print_node(child)

print_node(optimizer.root)

optimizing task e7a7c4d4-bbfd-47d1-9375-0e821032e122
****************************************************************************************************
-1 58082dd4aeee4a89870817fb11247611 1 0.0
****************************************************************************************************
0 762408de52714372bb777449b7f625db 1 0.0
main
The remaining steps should be optimized to directly address the user's goal of diagnosing and resolving schema lease errors on a single TiDB node using `tidb-ctl`. Specifically, the steps should include: 1) Adding a step to gather specific diagnostic commands and procedures using `tidb-ctl` to check schema versions and lease status on the affected node. 2) Modifying the vector_search step to focus on retrieving documentation or guides specifically related to using `tidb-ctl` for schema lease error diagnosis. 3) Enhancing the llm_generate step to synthesize the gathered information into a step-by-step guide for using `tidb-ctl` to diagnose and resolv

In [4]:
optimizer.optimize()

2025-02-18 20:13:43,323 - INFO - plan_mcts_optimizer.py:407 - selected node: 762408de52714372bb777449b7f625db(0)
2025-02-18 20:13:46,284 - INFO - engine.py:94 - Registered handler for instruction: calling
2025-02-18 20:13:46,284 - INFO - engine.py:94 - Registered handler for instruction: jmp
2025-02-18 20:13:46,285 - INFO - engine.py:94 - Registered handler for instruction: assign
2025-02-18 20:13:46,285 - INFO - engine.py:94 - Registered handler for instruction: reasoning
2025-02-18 20:13:46,674 - INFO - engine.py:408 - State loaded from commit 762408de52714372bb777449b7f625db
2025-02-18 20:13:46,675 - INFO - manager.py:446 - Update plan for Task ID e7a7c4d4-bbfd-47d1-9375-0e821032e122 from commit hash: 762408de52714372bb777449b7f625db to address the suggestion: The remaining steps should be optimized to directly address the user's goal of diagnosing and resolving schema lease errors on a single TiDB node using `tidb-ctl`. Specifically, the steps should include: 1) Adding a step to ga

In [5]:
answers = optimizer.sort_final_answers()
answers

2025-02-18 20:27:10,287 - INFO - _client.py:1038 - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


[{'commit_hash': '4da198e15cc44edc9326363d6e5c0cce',
  'score': 9.0,
  'final_answer': "### Step-by-Step Guide to Diagnosing and Resolving Schema Lease Errors on a Single TiDB Node\n\nSchema lease errors in TiDB can disrupt database operations, particularly those involving schema changes. This guide provides a comprehensive approach to diagnosing and resolving these errors using `tidb-ctl` commands and other troubleshooting techniques.\n\n#### Step 1: Verify Current Lease Configuration\n\n**Command**: `tidb-ctl lease`\n\n- **Purpose**: To check the current lease duration setting on the TiDB node.\n- **Example Output**:\n  ```\n  Lease: 45s\n  ```\n- **Interpretation**: The lease duration is set to 45 seconds. This setting impacts how quickly schema changes propagate across the cluster. Ensure this value is appropriate for your environment—typically longer in production for safety.\n\n#### Step 2: List and Monitor DDL Jobs\n\n**Command**: `tidb-ctl ddl jobs`\n\n- **Purpose**: To list on

In [6]:
from notebooks.tasks import save_best_plan_from_url

save_best_plan_from_url(task_id=task_id, commit_hash=answers[0]['commit_hash'])
# save_best_plan_from_url(task_id="2074ef3a-78d5-4d39-a753-5ba2c86fd0f8", commit_hash="b56ee2672ea2499eb9c36b0f180e89d9")

2025-02-18 20:28:07,105 - INFO - engine.py:64 - ThreadPoolExecutor has been shut down.
2025-02-18 20:28:07,105 - INFO - engine.py:64 - ThreadPoolExecutor has been shut down.


True